In [1]:
import altair as alt
import pandas as pd
import pickle
import os
import re

In [2]:
ddir = "../data/combined_transcripts_freeConv"
meta = pickle.load(open("../resources/meta.pkl", "rb"))
topics = pd.read_csv('../results/bert_probabilities_merged_85_new_names.csv', sep=',')
topic_names = topics.set_index('Topic')['CustomName']
prob_matrix = pd.read_csv("../results/prob_matrix.csv", sep=",")
prob_matrix["idx"] = list(range(prob_matrix.shape[0]))
prob_matrix.drop(columns=["25"], inplace=True)
all_docs = pickle.load(open("../resources/docs.pkl", "rb"))
personality_df = pd.read_csv("../materials/commgame_quest_scores.csv", sep=',')

## Minden kísérleti alanynak kiszámoljuk, hogy átlagban a 25 témáról mennyit beszélt. A diskurzusban az ő beszédfordulói milyen arányben szólnak a 25 topik mindegxyikéről. 

In [3]:
import numpy as np

discourse_matrix_mordor = []
discourse_matrix_gondor = []
discourse_idx = []

filenames = list(meta.values())
filenames = set([e.split(".txt")[0] for e in filenames])
for filename in filenames:
    file_meta = [(k,v) for k,v in meta.items() if filename in v]
    vals = [e[0] for e in file_meta]
    vals_mordor = [e[0] for e in file_meta if "mordor" in e[1]]
    vals_gondor = [e[0] for e in file_meta if "gondor" in e[1]]
    match = re.search(r'pair(\d+)', filename)
    if match:
        numeric_value = int(match.group(1))
    pm = prob_matrix.drop(columns=["idx"])
    disc_prob_matrix_mordor = prob_matrix[prob_matrix["idx"].isin(vals_mordor)]
    disc_prob_matrix_gondor = prob_matrix[prob_matrix["idx"].isin(vals_gondor)]
    disc_prob_matrix_mordor.drop(columns=["idx"], inplace=True)
    disc_prob_matrix_gondor.drop(columns=["idx"], inplace=True)
    
    m_mordor = np.mean(disc_prob_matrix_mordor.values, axis=0)
    m_gondor = np.mean(disc_prob_matrix_gondor.values, axis=0)
    discourse_matrix_mordor.append(m_mordor)
    discourse_matrix_gondor.append(m_gondor)
    discourse_idx.append(numeric_value)

/tmp/ipykernel_3954/4165049858.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  disc_prob_matrix_mordor.drop(columns=["idx"], inplace=True)
/tmp/ipykernel_3954/4165049858.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  disc_prob_matrix_gondor.drop(columns=["idx"], inplace=True)
/tmp/ipykernel_3954/4165049858.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  disc_prob_matrix_mordor.drop(columns=["idx"], inplace=True)
/tmp/ip

## A gondor és mondor alanyokat összerakjuk egy dataframebe. Az orszlopok a topikok, a sorok a beszélgetőpartnerek. Az értékek a valószínűségek. 2 plusz oszlop van: a páros száma és a labor neve (mordor vagy gondor).

In [4]:
discourse_matrix_mordor = np.asarray(discourse_matrix_mordor)
discourse_matrix_gondor = np.asarray(discourse_matrix_gondor)
discourse_df_mordor = pd.DataFrame(data=discourse_matrix_mordor,
                            index=list(range(discourse_matrix_mordor.shape[0])),
                            columns=list(range(25)))
discourse_df_mordor["PairNo"] = discourse_idx
discourse_df_gondor = pd.DataFrame(data=discourse_matrix_gondor,
                            index=list(range(discourse_matrix_gondor.shape[0])),
                            columns=list(range(25)))
discourse_df_gondor["PairNo"] = discourse_idx

In [5]:
discourse_df_mordor["LabName"] = "Mordor"
discourse_df_gondor["LabName"] = "Gondor"

In [6]:
discourse_df = pd.concat([discourse_df_mordor, discourse_df_gondor])
discourse_df

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,PairNo,LabName
0,0.187982,0.149602,0.056303,0.054014,0.055444,0.029347,0.004789,0.003234,0.003358,0.013444,...,0.013690,0.021677,0.008133,0.007632,0.011437,0.003410,0.023564,0.007061,114,Mordor
1,0.162960,0.224872,0.068464,0.021776,0.095728,0.009743,0.006084,0.004403,0.038960,0.011896,...,0.014695,0.009724,0.011365,0.010038,0.010499,0.004703,0.010894,0.008512,143,Mordor
2,0.200447,0.187475,0.045397,0.040569,0.015470,0.017903,0.004525,0.014140,0.008091,0.011498,...,0.026900,0.007626,0.007652,0.006720,0.009376,0.003639,0.009133,0.006277,156,Mordor
3,0.187880,0.210223,0.030323,0.024730,0.027417,0.009921,0.031805,0.010321,0.003775,0.026215,...,0.016297,0.009016,0.008590,0.032244,0.010724,0.004041,0.010783,0.007428,146,Mordor
4,0.237148,0.078539,0.035448,0.042038,0.036500,0.043392,0.004768,0.015056,0.036889,0.030718,...,0.014229,0.009903,0.008975,0.032893,0.011980,0.003447,0.011419,0.019897,181,Mordor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,0.095518,0.238256,0.066007,0.033341,0.022107,0.015924,0.029039,0.002772,0.027191,0.034871,...,0.006754,0.006121,0.005354,0.006312,0.007326,0.002985,0.006929,0.019278,175,Gondor
120,0.219672,0.135217,0.081259,0.030687,0.024174,0.015718,0.005566,0.004100,0.007506,0.013375,...,0.012233,0.012662,0.009906,0.009818,0.011867,0.010040,0.011898,0.008263,148,Gondor
121,0.262865,0.070495,0.184628,0.045085,0.021834,0.015539,0.006000,0.004077,0.004147,0.016235,...,0.026398,0.012728,0.022599,0.010703,0.014817,0.014730,0.014673,0.009493,128,Gondor
122,0.171340,0.154601,0.079294,0.044071,0.034358,0.013725,0.005146,0.003622,0.124235,0.011416,...,0.023220,0.008740,0.008013,0.008207,0.009665,0.003822,0.009650,0.007311,76,Gondor


## Megnézzük, melyik 3 témáról beszéltek a legtöbbet az alanyok.

In [7]:
selected_columns = discourse_df.iloc[:,0:25]
top_3_columns = selected_columns.apply(lambda row: row.nlargest(3).index.tolist(), axis=1)
top_3_columns_df = pd.DataFrame(top_3_columns.tolist(), columns=['Top1', 'Top2', 'Top3'])
top_3_columns_df

,Top1,Top2,Top3
0,0,1,15
1,1,0,4
2,0,1,2
3,1,0,14
4,0,1,5
...,...,...,...
243,1,10,0
244,0,1,2
245,0,2,1
246,0,1,8


## Megnézzük, hány darab topikról beszéltek több, mint 3%-ot az alanyok. Tetszés szerint változtathatjuk az arányt. Érdemes kísérletezni.

In [8]:
count_treshold = (discourse_df.iloc[:, 0:25] > 0.03).sum(axis=1)
count_treshold

0      7
1      6
2      5
3      6
4      9
      ..
119    7
120    6
121    5
122    7
123    6
Length: 248, dtype: int64